In [1]:
from langchain.text_splitter import MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

1. 데이터 로드
2. 텍스트 분할
3. 인덱싱(임베딩 후 인덱싱)
4. 검색(retrieval)

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# import subprocess
# import ocrmypdf
# from PIL import Image
# Image.MAX_IMAGE_PIXELS = None

# def lower_pdf_resolution(input_pdf_path, output_pdf_path, dpi=400):
#     try:
#         gs_command = [
#             "gswin64c.exe", # 또는 윈도우에서는 "gswin64c.exe" 등 Ghostscript 실행 파일 경로
#             "-sDEVICE=pdfwrite",
#             "-dCompatibilityLevel=1.4",
#             f"-dColorImageResolution={dpi}",
#             f"-dGrayImageResolution={dpi}",
#             f"-dMonoImageResolution={dpi}",
#             "-dDownsampleColorImages=true",
#             "-dDownsampleGrayImages=true",
#             "-dDownsampleMonoImages=true",
#             "-dNOPAUSE",
#             "-dBATCH",
#             f"-sOutputFile={output_pdf_path}",
#             input_pdf_path
#         ]
#         subprocess.run(gs_command, check=True)
#         print(f"PDF 해상도 조정 완료: {output_pdf_path} (목표 DPI: {dpi})")
#         return True
#     except FileNotFoundError:
#         print("Ghostscript가 설치되어 있지 않거나 경로에 없습니다. Ghostscript를 설치해주세요.")
#         return False
#     except subprocess.CalledProcessError as e:
#         print(f"Ghostscript 처리 중 오류 발생: {e}")
#         return False

# # 예시:

# if lower_pdf_resolution(input_file, preprocessed_file, dpi=300):
#     ocrmypdf.ocr(
#     preprocessed_file,
#     result_file,
#     language="kor",
#     deskew=True,
#     force_ocr=True,
#     max_image_mpixels=None,
#     image_dpi=300
# )

In [4]:
# input_pdf = "datasets/manual.pdf"

input_pdf = "datasets/여비산정기준표.pdf"
preprocessed_file = "datasets/ocr_output.pdf"
result_file = "datasets/ocr_output_result.pdf",

In [5]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode

pipeline_options = PdfPipelineOptions(
    do_ocr=True,
    do_table_structure = True,
    do_text_extraction=True,
)
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

doc = doc_converter.convert(input_pdf)

markdown_text = doc.document.export_to_markdown()
# print(markdown_text)
#읽지 못하는 텍스트와 사진 처리방법 고민

In [6]:
# import pdfplumber
# import pandas as pd

# def convert_pdf_with_pdfplumber_to_markdown_text(input_pdf_path: str) -> str:
#     """
#     PDF의 텍스트와 테이블을 추출하여 하나의 마크다운 문자열로 반환합니다.
#     """
#     markdown_text = ""
    
#     with pdfplumber.open(input_pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages):
#             markdown_text += f"## Page {page_num + 1}\n\n"

#             # 1) 텍스트 추출
#             text = page.extract_text()
#             if text:
#                 markdown_text += text.strip() + "\n\n"

#             # 2) 테이블 추출
#             tables = page.extract_tables()
#             for table_idx, table in enumerate(tables):
#                 try:
#                     df = pd.DataFrame(table[1:], columns=table[0])
#                     table_md = df.to_markdown(index=False)
#                     markdown_text += f"### Table {page_num + 1}-{table_idx + 1}\n\n"
#                     markdown_text += table_md + "\n\n"
#                 except Exception as e:
#                     markdown_text += f"### Table {page_num + 1}-{table_idx + 1} (Error parsing table)\n\n"

#             markdown_text += "---\n\n"
    
#     return markdown_text

# markdown_text = convert_pdf_with_pdfplumber_to_markdown_text(input_pdf)

In [7]:
# import fitz  # PyMuPDF
# import pandas as pd
# import pytesseract
# from PIL import Image
# import io

# def convert_pdf_with_fitz_to_markdown_text(input_pdf_path: str) -> str:
#     """
#     fitz(PyMuPDF) + pytesseract를 사용해 PDF 텍스트와 이미지 기반 테이블을 추출하여 마크다운 형식으로 반환
#     """
#     markdown_text = ""
#     doc = fitz.open(input_pdf_path)

#     for page_num, page in enumerate(doc):
#         markdown_text += f"## Page {page_num + 1}\n\n"

#         # 1) 페이지 텍스트 추출
#         text = page.get_text()
#         if text.strip():
#             markdown_text += text.strip() + "\n\n"

#         # 2) 페이지를 이미지로 렌더링
#         pix = page.get_pixmap(dpi=300)
#         img = Image.open(io.BytesIO(pix.tobytes("png")))

#         # 3) pytesseract를 사용해 이미지에서 테이블 추출 시도 (OCR 기반)
#         try:
#             table_data = pytesseract.image_to_string(img, config="--psm 6", lang="kor+eng")
#             if table_data.strip():
#                 markdown_text += f"### Table (OCR from image)\n\n"
#                 markdown_text += "```\n" + table_data.strip() + "\n```\n\n"
#         except Exception as e:
#             markdown_text += f"### Table OCR Error: {str(e)}\n\n"

#         markdown_text += "---\n\n"

#     doc.close()
#     return markdown_text

# markdown_text = convert_pdf_with_fitz_to_markdown_text(input_pdf)

In [8]:
import pdfplumber
import pandas as pd

def table_chunker(pdf_path: str):
    chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                # table: 2D 리스트 (헤더 포함)
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                except:
                    df = pd.DataFrame(table)
                md_table = df.to_markdown(index=False)
                chunks.append({
                    "text": md_table,
                    "metadata": {
                        "type": "table",
                        "page": page.page_number
                    }
                })
    return chunks

# 사용 예시
table_chunks = table_chunker(input_pdf)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [9]:
from docling.chunking import HybridChunker

chunker = HybridChunker(
    tokenizer='BAAI/bge-m3',
    max_token_length=256,
    overlap=50
)

chunks = list(chunker.chunk(doc.document))



# chunks는 청커로 생성된 문서 조각들
# documents = [Document(page_content=chunk.text, metadata={"source": chunk.meta}) for chunk in chunks]

In [10]:
text_documents = []
for chunk in chunks:
    text = chunk.text

    # page_no를 추출 (여러 Prov 아이템 중 첫 번째의 page_no를 사용하거나, set으로 처리)
    page_nos = sorted(
        set(
            prov.page_no
            for item in chunk.meta.doc_items
            for prov in item.prov
            if hasattr(prov, "page_no")
        )
    ) if hasattr(chunk.meta, "doc_items") else []

    # 가장 근접한 헤딩(섹션 이름) 추출 (예: 마지막 요소)
    headings = chunk.meta.headings if hasattr(chunk.meta, "headings") else []
    section = headings[-1] if headings else None

    metadata = {
        "source": input_pdf,
        "page_numbers": page_nos,
        "section": section
    }
    # (3) LangChain Document 생성
    text_documents.append(Document(page_content=text, metadata=metadata))

In [11]:
# 1. table_chunks를 Document 형식으로 변환
table_documents = [
    Document(
        page_content=chunk["text"],
        metadata={
            "type": "table",
            "page": chunk["metadata"]["page"],
            "source": input_pdf
        }
    ) for chunk in table_chunks
]

# 3. 두 Document 리스트 합치기
all_documents = table_documents + text_documents

In [12]:
# splitter = MarkdownTextSplitter(
#     chunk_size=800,
#     chunk_overlap=100
# ) #잘 안돼면 512, 768, 1024 등으로 변경
# chunks = splitter.create_documents([markdown_text])

# # 3. Document 객체 생성
# documents = [Document(page_content=chunk.page_content, metadata={"source": input_pdf}) for chunk in chunks]

In [15]:
# embedding_model = HuggingFaceEmbeddings(
#     model_name='BAAI/bge-m3',
#     model_kwargs={'device':device},
#     encode_kwargs={'normalize_embeddings':True},
# )

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

vectorstore = FAISS.from_documents(
    documents=all_documents,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever()

In [ ]:
# results = vectorstore.similarity_search_with_score(
#     query="교수가 미국으로 4일 출장을 다녀왔을 때 받을 수 있는 숙박비는?", 
#     k=5  # top-k 개수
# )

# for i, (doc, score) in enumerate(results):
#     print(f"Document {i+1}: (Score: {score:.4f})\n{doc.page_content}\n")


In [16]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=8)

# 문서 압축 검색기 초기화
compression_reranker = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
# query = "미국은 가 지역에 속해?"

# # 1. Top-k 유사도 문서 + 점수 가져오기
# raw_results = retriever.vectorstore.similarity_search_with_score(query, k=5)
# docs, scores = zip(*raw_results)

# # 2. 문서 압축 (요약/필터링)
# compressed_docs = compressor.compress_documents(list(docs), query=query)

# # 3. 출력
# for i, (original, score) in enumerate(zip(docs, scores)):
#     compressed = next((cd for cd in compressed_docs if cd.page_content in original.page_content), None)
#     print(f"\nDoc {i+1} (Score: {score:.4f}):")
#     print(f"- Original:\n{original.page_content[:300]}")
#     if compressed:
#         print(f"- Compressed:\n{compressed.page_content[:300]}")


In [18]:
from langchain_ollama import OllamaLLM

# llm = OllamaLLM(model="llama3.1:latest")
# llm = OllamaLLM(model="qwen3:latest")
llm = OllamaLLM(model="qwen3:4b")
# llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
You must answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:
""")

rag_chain = (
    {'context': compression_reranker | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
question = "교수가 미국으로 4일 출장을 다녀왔을 때 받을 수 있는 총 숙박비는?"
result = rag_chain.invoke(question)
print(result)

<think>
Okay, let's try to figure out how much the professor can get for accommodation costs when they went on a 4-day business trip to the US.

First, I need to check the relevant information from the provided tables. The question is about the accommodation cost for a professor traveling to the US. Looking at the "체 재 비" (accommodation cost) section, there's a table that categorizes costs based on the region. The "가" region includes the US, as per the list: "가 : 스위스,이스라엘, 아이슬 란드,노르웨이, 덴마크, 아일랜드,오스트레일리아, 뉴질랜드,룩셈부르크,스웨 덴, 영국, 핀란드, 캐나다, 미국, 네덜란드,벨기에, 일본, 오스트리아, 프랑스, 독일,이탈리아, 홍콩,싱가포르, 러시아(모스크바),중국(베이징, 상하이,선전,광저우)". So the US is under the "가" category.

Looking at the "체 재 비" table, for the "가" region, the accommodation cost per night is $220. However, there's also a note about the duration of stay. The first table mentions that if someone stays in the same place for a long time, the costs are reduced. But the question is about a 4-day trip. Wait, but the question is about the total accom

In [ ]:
# #pdf내의 표를 저장함
# from pathlib import Path

# def extract_and_export_tables(input_path, output_dir):
#     output_dir.mkdir(parents=True, exist_ok=True)
    
#     doc_converter = DocumentConverter()
    
#     conv_res = doc_converter.convert(input_path)
#     doc_filename = conv_res.input.file.stem
    
#     for table_ix, table in enumerate(conv_res.document.tables):
#         table_df = table.export_to_dataframe()
        
#         csv_filename = output_dir / f"{doc_filename}-table-{table_ix+1}.csv"
#         table_df.to_csv(csv_filename)
        
#         html_filename = output_dir / f"{doc_filename}-table-{table_ix+1}.html"
#         with html_filename.open("w") as fp:
#             fp.write(table.export_to_html())
            
#     table_count = len(conv_res.document.tables) 
    
#     return table_count


# output_dir = Path("output")
# output_dir.mkdir(parents=True, exist_ok=True)

# table_count = extract_and_export_tables(filepath, output_dir)
# print("success")

In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain_docling.loader import ExportType

# filepath = 'datasets/여비산정기준표.pdf'
# embedding_model = 'BAAI/bge-m3'
# reranking_model = 'Alibaba-NLP/gte-multilingual-reranker-base'
# export_type = ExportType.DOC_CHUNKS

# prompt = PromptTemplate.from_template("""
# You are an assistant for question-answering tasks. 
# Use the following pieces of retrieved context to answer the question.
# If the user asks for a simple answer, summarize the key points.
# If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
# You must answer in Korean.

# #Context: 
# {context}

# #Question:
# {question}

# #Answer:
# """)

# from langchain_docling import DoclingLoader
# from docling.chunking import HybridChunker

# loader = DoclingLoader(
#     file_path = filepath,
#     export_type = ExportType.DOC_CHUNKS,
#     chunker = HybridChunker(tokenizer=embedding_model)
# )

# docs = loader.load()

# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_milvus import Milvus

# MUIVUS_URI = "./milvus_example.db"

# embedding = HuggingFaceEmbeddings(
#     model_name=embedding_model,
#     model_kwargs={'device':device,
#                   'trust_remote_code':True}
# )

# vectorstore = Milvus.from_documents(
#     documents=docs,
#     embedding=embedding,
#     collection_name="docling_transformer",
#     connection_args={
#         "url":MUIVUS_URI,
#         "db_name":" edu"
#     },
#     index_params={
#         "index_type":"FLAT",
#         "metric_type": "cosine"
#     },
#     drop_old=True
# )